In [1]:
import pandas as pd

data = pd.read_csv(r"C:\Users\RICH\Downloads\rt-iot2022\RT_IOT2022 - 副本.csv")

print(data.shape)

(123117, 85)


In [2]:
# data = pd.read_csv(r"C:\Users\RICH\Downloads\rt-iot2022\RT_IOT2022 - 副本.csv")

# 获取非数值列
columns = ['proto', 'service', 'Attack_type']
dictionary = []
for col in columns:
    print(col)
    col_data = {}
    for i, val in enumerate(data[col].unique()):
            col_data[val] = i
    dictionary.append(col_data)
print(dictionary)

proto
service
Attack_type
[{'tcp': 0, 'udp': 1, 'icmp': 2}, {'mqtt': 0, '-': 1, 'http': 2, 'dns': 3, 'ntp': 4, 'ssl': 5, 'dhcp': 6, 'irc': 7, 'ssh': 8, 'radius': 9}, {'MQTT_Publish': 0, 'Thing_Speak': 1, 'Wipro_bulb': 2, 'ARP_poisioning': 3, 'DDOS_Slowloris': 4, 'DOS_SYN_Hping': 5, 'Metasploit_Brute_Force_SSH': 6, 'NMAP_FIN_SCAN': 7, 'NMAP_OS_DETECTION': 8, 'NMAP_TCP_scan': 9, 'NMAP_UDP_SCAN': 10, 'NMAP_XMAS_TREE_SCAN': 11}]


In [3]:
for d in dictionary:
    print(d)

{'tcp': 0, 'udp': 1, 'icmp': 2}
{'mqtt': 0, '-': 1, 'http': 2, 'dns': 3, 'ntp': 4, 'ssl': 5, 'dhcp': 6, 'irc': 7, 'ssh': 8, 'radius': 9}
{'MQTT_Publish': 0, 'Thing_Speak': 1, 'Wipro_bulb': 2, 'ARP_poisioning': 3, 'DDOS_Slowloris': 4, 'DOS_SYN_Hping': 5, 'Metasploit_Brute_Force_SSH': 6, 'NMAP_FIN_SCAN': 7, 'NMAP_OS_DETECTION': 8, 'NMAP_TCP_scan': 9, 'NMAP_UDP_SCAN': 10, 'NMAP_XMAS_TREE_SCAN': 11}


In [4]:
# 更新csv并保存
for i, col in enumerate(columns):
    data[col] = data[col].map(dictionary[i])

data.to_csv(r"RT_IOT2022_num.csv", index=False)

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取CSV文件
data = pd.read_csv("RT_IOT2022_num.csv")

# 第一次划分：分出训练集(70%)和临时测试集(30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    data.drop('Attack_type', axis=1),  # 特征
    data['Attack_type'],               # 标签
    test_size=0.3,                     # 30%作为临时测试集
    random_state=42                    # 设置随机种子，确保结果可复现
)

# # 第二次划分：将临时测试集分为验证集(15%)和测试集(15%)
# X_val, X_test, y_val, y_test = train_test_split(
#     X_temp, y_temp,
#     test_size=0.5,                     # 临时测试集对半分
#     random_state=42
# )

# 重新组合特征和标签
train_data = pd.concat([X_train, y_train], axis=1)
# val_data = pd.concat([X_val, y_val], axis=1)
# test_data = pd.concat([X_test, y_test], axis=1)
test_data = pd.concat([X_temp, y_temp], axis=1)

# 保存到CSV文件
train_data.to_csv('train.csv', index=False)
# val_data.to_csv('val.csv', index=False)
# test_data.to_csv('test.csv', index=False)
test_data.to_csv('test.csv', index=False)
# 打印数据集大小
print(f"训练集大小: {len(train_data)}")
# print(f"验证集大小: {len(val_data)}")
print(f"测试集大小: {len(test_data)}")

训练集大小: 86181
测试集大小: 36936


In [ ]:
import pandas as pd
import numpy as np

def validate_data_integrity():
    # 读取所有数据集
    original = pd.read_csv("RT_IOT2022_num.csv")
    train = pd.read_csv('train.csv')
    val = pd.read_csv('val.csv')
    test = pd.read_csv('test.csv')
    split_combined = pd.concat([train, val, test])
    
    checks = []
    
    # 1. 检查样本总数
    checks.append(f"原始数据数量: {len(original)}")
    checks.append(f"划分后总数: {len(split_combined)}")
    checks.append(f"数量是否一致: {len(original) == len(split_combined)}")
    
    # 2. 检查数值范围
    for col in original.columns:
        orig_range = (original[col].min(), original[col].max())
        split_range = (split_combined[col].min(), split_combined[col].max())
        checks.append(f"\n列 {col} 的范围:")
        checks.append(f"原始数据: {orig_range}")
        checks.append(f"划分后: {split_range}")
    
    # 3. 检查数据类型
    checks.append("\n数据类型对比:")
    checks.append(f"原始数据:\n{original.dtypes}")
    checks.append(f"划分后:\n{split_combined.dtypes}")
    
    # 4. 检查基本统计量
    checks.append("\n基本统计量对比:")
    checks.append("\n原始数据描述:")
    checks.append(f"{original.describe()}")
    checks.append("\n划分后数据描述:")
    checks.append(f"{split_combined.describe()}")
    
    # 5. 检查唯一值
    for col in original.columns:
        orig_unique = set(original[col].unique())
        split_unique = set(split_combined[col].unique())
        checks.append(f"\n列 {col} 的唯一值:")
        checks.append(f"原始数据唯一值数量: {len(orig_unique)}")
        checks.append(f"划分后唯一值数量: {len(split_unique)}")
        checks.append(f"是否完全相同: {orig_unique == split_unique}")
    
    return "\n".join(map(str, checks))

# 执行验证并打印结果
print(validate_data_integrity())

In [1]:
# 构建分类模型
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

# 加载数据
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('val.csv')
test_data = pd.read_csv('test.csv')

# 准备特征和标签
X_train = train_data.drop('Attack_type', axis=1)
y_train = train_data['Attack_type']
X_val = val_data.drop('Attack_type', axis=1)
y_val = val_data['Attack_type']

# 标准化数据
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 初始化模型
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100)
    # 'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    # 'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    # 'SVM': SVC(kernel='rbf', random_state=42)
}

# 训练和评估模型
results = {}
for name, model in models.items():
    print(f"\n训练 {name}...")
    model.fit(X_train_scaled, y_train)
    val_pred = model.predict(X_val_scaled)
    accuracy = accuracy_score(y_val, val_pred)
    results[name] = accuracy
    print(f"{name} 验证集准确率: {accuracy:.4f}")
    print("\n分类报告:")
    print(classification_report(y_val, val_pred))

# 选择最佳模型
best_model = max(results.items(), key=lambda x: x[1])
print(f"\n最佳模型是 {best_model[0]}，准确率为 {best_model[1]:.4f}")

# 绘制混淆矩阵
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.figure(figsize=(10, 8))
best_predictions = models[best_model[0]].predict(X_val_scaled)
cm = confusion_matrix(y_val, best_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'{best_model[0]} 混淆矩阵')
plt.xlabel('预测标签')
plt.ylabel('真实标签')
plt.show()


训练 RandomForest...


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# 自定义数据集类
class IOTDataset(Dataset):
    def __init__(self, data_path):
        df = pd.read_csv(data_path)
        df = df.drop('Unnamed: 0', axis=1)
        self.X = df.drop('Attack_type', axis=1).values
        self.y = df['Attack_type'].values
        
        # 标准化特征
        scaler = StandardScaler()
        self.X = scaler.fit_transform(self.X)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.X[idx]), torch.LongTensor([self.y[idx]])[0]

# 定义神经网络模型
class IOTClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(IOTClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size//2, num_classes)
        )
    
    def forward(self, x):
        return self.model(x)

# 设置参数
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 83  # 根据实际特征数量调整
hidden_size = 128
num_classes = 12  # 根据实际类别数量调整
num_epochs = 50
batch_size = 32
learning_rate = 0.001

# 加载数据
print("加载数据...")
train_dataset = IOTDataset('train.csv')
val_dataset = IOTDataset('val.csv')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# 初始化模型
model = IOTClassifier(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model_path = r"D:\projects\py\RT-IoT2022\iot_classifier.pth"
model.load_state_dict(torch.load(model_path,weights_only=True))
# 训练循环
print("开始训练...")
for epoch in tqdm(range(41,num_epochs), desc='训练进度'):
    model.train()
    train_loss = 0
    train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    
    for batch_X, batch_y in train_pbar:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    # 验证
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc='验证', leave=False)
        for batch_X, batch_y in val_pbar:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
            
            _, predicted = outputs.max(1)
            total += batch_y.size(0)
            correct += predicted.eq(batch_y).sum().item()
            val_pbar.set_postfix({'acc': f'{100.*correct/total:.2f}%'})
    
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f'Train Loss: {train_loss/len(train_loader):.4f}')
    print(f'Val Loss: {val_loss/len(val_loader):.4f}')
    print(f'Val Accuracy: {100.*correct/total:.2f}%')

print("训练完成!")
# 保存模型
torch.save(model.state_dict(), 'iot_classifier.pth')

加载数据...


AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)